In [1]:
# The BN254 Elliptic curve E(Fp) : y^2 = x^3 + 3 
p = 21888242871839275222246405745257275088696311157297823662689037894645226208583
Fp = GF(p)
E = EllipticCurve(Fp, [0,3])
r = E.order()
Fr = GF(r)

# Field extension of degree 12
Fp12.<a> = GF(p^12)
RFp12.<T> = PolynomialRing(Fp12)

# We define an twist of E over an extension field
j = (T^2 + 1).roots(ring=Fp12, multiplicities=0)[0]
E2 = EllipticCurve(Fp12, [0,3])
E2_twist = EllipticCurve(Fp12, [0, 0x2b149d40ceb8aaae81be18991be06ac3b5b4c5e559dbefa33267e6dc24a138e5 + 266929791119991161246907387137283842545076965332900288569378510910307636690*j])
phi = E2_twist.isomorphism_to(E2)

# The pairing over E
def e(P,Q):
    # embedding degree
    k = 12
    # trace
    t = p + 1 - r
    return P.ate_pairing(phi(Q), r, k, t, p)

# The generators G1, G2 of order r
G1 = E2([1, 2])
G2 = E2_twist([0x1800deef121f1e76426a00665e5c4479674322d4f75edadd46debd5cd992f6ed+0x198e9393920d483a7260bfb731fb5d25f1aa493335a9e71297e485b7aef312c2*j, 0x12c85ea5db8c6deb4aab71808dcb408fe3d1e7690c43d37b4ce6cc0166fa7daa + 0x090689d0585ff075ec9e99ad690c3395bc4b313370b38ef355acdadcd122975b*j])
assert r*G1 == E2(0)
assert phi(r*G2) == E2(0)

# We check the pairing is a bilinear map
assert e(2*G1,3*G2) == e(3*G1,2*G2) == e(1*G1,2*G2)*e(2*G1,2*G2) == e(G1,G2)^6

# The polynomial Ring over Fr
R.<x> = PolynomialRing(Fr)

# Generates a random polynomial of degree less than n
def rand_poly(n):
    return R([R.base_ring().random_element() for _ in range(n)])

In [2]:
# Generates a SRS from the curve's generators G1 and G2
def setup(n):
    s = Fr.random_element()
    SRS = [s^i*G1 for i in range(n)]
    s_G2 = s*G2
    return SRS, s_G2
    
# Commits to a polynomial
def commit(f, SRS):
    assert f.degree() < len(SRS)
    return sum([ai*Gi for ai, Gi in zip(f.coefficients(), SRS)])

# Computes an evaluation proof for f in z
def prove(f, z, SRS):
    c = f(z)
    g = R( (f - c)/ (x-z) )
    Cg = commit(g, SRS)
    return Cg, c

# Verifies an evaluation proof (Cg, z, c) with respect to a commitment Cf
def verify(Cf, Cg, c, z, s_G2):
    return e(Cf, G2) == e(G1,G2)^c * e(Cg, s_G2-z*G2)

In [3]:
# The degree bound
n = 100

# Setup
SRS, s_G2 = setup(n)

# We generate a random polynomial
f = rand_poly(n)

# The Prover commits to f
Cf = commit(f, SRS)

# The Verifier picks a random evaluation point z and sends it to the Prover
z = Fr.random_element()

# The Prover computes an evaluation proof for f in z
Cg, c = prove(f, z, SRS)

# The Verifier checks the evaluation proof
assert verify(Cf, Cg, c, z, s_G2)